In [11]:
import numpy as np
import torch 
from torch import tensor
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f">> The hardware running is {dev}.")
a = np.linspace(-5, 5, num=10)
print(a)

b = torch.linspace(start=-5, end=5, steps=10)
print(b)

>> The hardware running is cuda:0.
[-5.         -3.88888889 -2.77777778 -1.66666667 -0.55555556  0.55555556
  1.66666667  2.77777778  3.88888889  5.        ]
tensor([-5.0000, -3.8889, -2.7778, -1.6667, -0.5556,  0.5556,  1.6667,  2.7778,
         3.8889,  5.0000])


In [38]:
from scipy import constants as const
lenA = const.angstrom  # 埃A
eps0 = const.epsilon_0  # 真空电容率
ec = const.elementary_charge  # 元电荷
eV = const.physical_constants["electron volt"][0]  # 电子伏特

c = torch.linspace(start=-5*lenA, end=5*lenA, steps=10).to(dev)
print(c)

tensor([-5.0000e-10, -3.8889e-10, -2.7778e-10, -1.6667e-10, -5.5556e-11,
         5.5556e-11,  1.6667e-10,  2.7778e-10,  3.8889e-10,  5.0000e-10],
       device='cuda:0')


In [20]:
def Laplacian(x_step, x_num):
    """
    --------------------------------------------
    Note:
        x_step is the duration or interval of x axis

    --------------------------------------------
    Return:
        二阶导数的有限差分矩阵表示 
    """
    D_2st = (
        torch.diag(tensor([1] * (x_num - 1)), diagonal=1)
        + torch.diag(tensor([1] * (x_num - 1)), diagonal=-1)
        + torch.diag(tensor([-2] * x_num))
    )
    return D_2st / (x_step**2)

Laplacian(1, 4)

tensor([[-2.,  1.,  0.,  0.],
        [ 1., -2.,  1.,  0.],
        [ 0.,  1., -2.,  1.],
        [ 0.,  0.,  1., -2.]])

In [21]:
def Laplacian(x_step, x_num):
    """x_step is the duration or interval of x axis"""
    D_2st = (
        np.diag([1] * (x_num - 1), k=1)
        + np.diag([1] * (x_num - 1), k=-1)
        + np.diag([-2] * x_num)
    )
    return D_2st / (x_step**2)  # 2阶微分的差分格式

Laplacian(1, 4)

array([[-2.,  1.,  0.,  0.],
       [ 1., -2.,  1.,  0.],
       [ 0.,  1., -2.,  1.],
       [ 0.,  0.,  1., -2.]])

In [23]:
mat_np = Laplacian(1, 4)
np.linalg.eig(mat_np)

(array([-3.61803399, -2.61803399, -0.38196601, -1.38196601]),
 array([[ 0.37174803,  0.60150096, -0.37174803, -0.60150096],
        [-0.60150096, -0.37174803, -0.60150096, -0.37174803],
        [ 0.60150096, -0.37174803, -0.60150096,  0.37174803],
        [-0.37174803,  0.60150096, -0.37174803,  0.60150096]]))

In [26]:
w, v = torch.linalg.eigh(tensor(mat_np))
print(w)
print(v)
torch.sort()

tensor([-3.6180, -2.6180, -1.3820, -0.3820], dtype=torch.float64)
tensor([[ 0.3717, -0.6015, -0.6015, -0.3717],
        [-0.6015,  0.3717, -0.3717, -0.6015],
        [ 0.6015,  0.3717,  0.3717, -0.6015],
        [-0.3717, -0.6015,  0.6015, -0.3717]], dtype=torch.float64)


In [30]:
m = tensor(mat_np)
m.diag()

tensor([-2., -2., -2., -2.], dtype=torch.float64)

In [34]:
def hydrogen_potential(x):
    """
    --------------------------------------------
    Note:
        类氢原子都是向心势 central potential
    """
    return -(ec**2) / (4 * torch.pi * eps0 * torch.abs(x))

xs = torch.linspace(start=-5*lenA, end=5*lenA, steps=10).to(dev)

hydrogen_potential(xs)

tensor([-4.6142e-19, -5.9325e-19, -8.3055e-19, -1.3842e-18, -4.1527e-18,
        -4.1527e-18, -1.3842e-18, -8.3055e-19, -5.9325e-19, -4.6142e-19],
       device='cuda:0')

In [36]:
def hydrogen_potential(x):
    """central potential"""
    return -(ec**2) / (4 * np.pi * eps0 * np.abs(x))

xs = np.linspace(-5*lenA, 5*lenA, 10)
hydrogen_potential(xs)

array([-4.61415510e-19, -5.93248513e-19, -8.30547919e-19, -1.38424653e-18,
       -4.15273959e-18, -4.15273959e-18, -1.38424653e-18, -8.30547919e-19,
       -5.93248513e-19, -4.61415510e-19])

In [42]:
def double_well(x, start0=-1, start1=1, width=1, magnitude=-1):
    """
    --------------------------------------------
    Description:
        双势阱
    """
    s0 = start0 * lenA
    s1 = start1 * lenA
    w = width * lenA
    m = magnitude * eV
    if width > 0:
        if start0 + width <= start1 or start1 + width <= start0:
            return m * (
                torch.heaviside(x - s0, tensor([0.5]))
                - torch.heaviside(x - s0 - w, tensor([0.5]))
                + torch.heaviside(x - s1, tensor([0.5]))
                - torch.heaviside(x - s1 - w, tensor([0.5]))
            )
        else:
            return "Not double_well !"
    else:
        return "Not double_well !"

double_well(tensor([1,2]))

tensor([-0., -0.])

In [40]:
tensor([1,2])-2

tensor([-1,  0])